In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.metrics import auc
from sklearn import preprocessing

In [2]:
# Please replace with extracted features from feature_extraction_cnn.ipynb or feature_extraction_vit.ipynb.
model_path = '/viscam/u/joycj/geoclidean/model_benchmarks/vit/'

train_pos_low = pickle.load(open(model_path + 'train_pos_low.p', 'rb'))
train_pos_high = pickle.load(open(model_path + 'train_pos_high.p', 'rb'))

test_pos_low = pickle.load(open(model_path + 'test_pos_low.p', 'rb'))
test_pos_high = pickle.load(open(model_path + 'test_pos_high.p', 'rb'))

test_neg_low_hard = pickle.load(open(model_path + 'test_neg_low_hard.p', 'rb'))
test_neg_high_hard = pickle.load(open(model_path + 'test_neg_high_hard.p', 'rb'))

test_neg_low_easy = pickle.load(open(model_path + 'test_neg_low_easy.p', 'rb'))
test_neg_high_easy = pickle.load(open(model_path + 'test_neg_high_easy.p', 'rb'))


In [3]:
def get_diff_to_mean_wug(train_wugs, wugs, not_wugs_close, not_wugs_far):
    mean_train_wugs = np.squeeze(np.array(train_wugs))
    mean_train_wugs = np.mean(mean_train_wugs, 0)
    
    wugs_diff = []
    for e in wugs:
        wugs_diff.append(np.sum(np.abs(e - mean_train_wugs)))

    not_wugs_close_diff = []
    for e in not_wugs_close:
        not_wugs_close_diff.append(np.sum(np.abs(e - mean_train_wugs)))
        
    not_wugs_far_diff = []
    for e in not_wugs_far:
        not_wugs_far_diff.append(np.sum(np.abs(e - mean_train_wugs)))
    
    return wugs_diff, not_wugs_close_diff, not_wugs_far_diff

def tpr_fpr_for_threshold(wug, not_wug, threshold):
    pos = 0
    for d in wug:
        if d <= threshold:
            pos += 1
    tpr = pos / len(wug)
    
    neg_wrong = 0
    for d in not_wug:
        if d <= threshold:
            neg_wrong += 1
    fpr = neg_wrong / len(not_wug)
    
    return tpr, fpr

def acc_for_threshold(wug, not_wug, threshold):
    total = 0
    pos = 0
    for d in wug:
        total += 1
        if d <= threshold:
            pos += 1
            
    neg = 0
    for d in not_wug:
        total += 1
        if d > threshold:
            neg += 1    
    return (pos + neg) / total

def get_auc(wugs_diff, not_wugs_diff):
    all_diffs = sorted(wugs_diff + not_wugs_diff)
    tprs = []
    fprs = []
    ts = all_diffs
    for t in ts:
        tpr, fpr = tpr_fpr_for_threshold(wugs_diff, not_wugs_diff, t)
        tprs.append(tpr)
        fprs.append(fpr)
    curr_auc = auc(fprs, tprs)
    return curr_auc, fprs, tprs
    
def eval_for_task(train_pos_low, train_pos_high, test_pos_low, test_pos_high, test_neg_low_hard, test_neg_high_hard, test_neg_low_easy, test_neg_high_easy):
    
    test_pos_low_diff, test_neg_low_hard_diff, test_neg_low_easy_diff = get_diff_to_mean_wug(train_pos_low, test_pos_low, test_neg_low_hard, test_neg_low_easy)
    auc_low_hard, _, _, = get_auc(test_pos_low_diff, test_neg_low_hard_diff)
    auc_low_easy, _, _, = get_auc(test_pos_low_diff, test_neg_low_easy_diff)
    
    test_pos_high_diff, test_neg_high_hard_diff, test_neg_high_easy_diff = get_diff_to_mean_wug(train_pos_high, test_pos_high, test_neg_high_hard, test_neg_high_easy)
    auc_high_hard, _, _, = get_auc(test_pos_high_diff, test_neg_high_hard_diff)
    auc_high_easy, _, _, = get_auc(test_pos_high_diff, test_neg_high_easy_diff)
    
    return auc_low_hard, auc_low_easy, auc_high_hard, auc_high_easy



In [4]:
def get_normalized_diff_to_mean_wug(train_wugs, wugs, not_wugs_close, not_wugs_far):
    mean_train_wugs = np.squeeze(np.array(train_wugs))
    mean_train_wugs = np.mean(mean_train_wugs, 0)
    
    wugs_diff = []
    for e in wugs:
        wugs_diff.append(np.sum(np.abs(e - mean_train_wugs)))

    not_wugs_close_diff = []
    for e in not_wugs_close:
        not_wugs_close_diff.append(np.sum(np.abs(e - mean_train_wugs)))
        
    not_wugs_far_diff = []
    for e in not_wugs_far:
        not_wugs_far_diff.append(np.sum(np.abs(e - mean_train_wugs)))
        
    joint = wugs_diff + not_wugs_close_diff    
    normalized = (joint - np.min(joint))/np.ptp(joint)
    normalized_wugs_diff_by_close = list(normalized[:5])
    normalized_not_wugs_close_diff = list(normalized[5:])
    
    joint = wugs_diff + not_wugs_far_diff    
    normalized = (joint - np.min(joint))/np.ptp(joint)
    normalized_wugs_diff_by_far = list(normalized[:5])
    normalized_not_wugs_far_diff = list(normalized[5:])
    
    return normalized_wugs_diff_by_close, normalized_wugs_diff_by_far, normalized_not_wugs_close_diff, normalized_not_wugs_far_diff

def get_max_t(wugs_diff, not_wugs_diff, ts):
    mean_accs_for_t = []
    for t in ts:
        accs_for_tasks = []
        for (pos, neg) in zip(wugs_diff, not_wugs_diff):
            accs_for_tasks.append(acc_for_threshold(pos, neg, t))
        mean_accs_for_t.append(sum(accs_for_tasks) / len(accs_for_tasks))
    max_t_arg = np.argmax(mean_accs_for_t)
    return ts[max_t_arg]

def eval_max_acc(train_pos, test_pos, test_neg_hard, test_neg_easy):
    # get all normalized thresholds
    all_potential_ts = []
    wugs_diff = []
    not_wugs_diff = []
    for (p, tp, tn_hard, tn_easy) in zip(train_pos, test_pos, test_neg_hard, test_neg_easy):
        normalized_wugs_diff_by_close, normalized_wugs_diff_by_far, normalized_not_wugs_close_diff, normalized_not_wugs_far_diff = get_normalized_diff_to_mean_wug(p, tp, tn_hard, tn_easy)
        all_potential_ts += normalized_wugs_diff_by_close
        all_potential_ts += normalized_wugs_diff_by_far
        all_potential_ts += normalized_not_wugs_close_diff
        all_potential_ts += normalized_not_wugs_far_diff
        
        wugs_diff.append(normalized_wugs_diff_by_close)
        not_wugs_diff.append(normalized_not_wugs_close_diff)
        wugs_diff.append(normalized_wugs_diff_by_far)
        not_wugs_diff.append(normalized_not_wugs_far_diff)
        
    assert len(all_potential_ts) == 74 * 10
    assert len(wugs_diff) == 74 * 1
    
    # get max t
    max_t = get_max_t(wugs_diff, not_wugs_diff, all_potential_ts)
    
    # get accuracies
    hard_accs = []
    easy_accs = []
    for (p, tp, tn_hard, tn_easy) in zip(train_pos, test_pos, test_neg_hard, test_neg_easy):
        normalized_wugs_diff_by_close, normalized_wugs_diff_by_far, normalized_not_wugs_close_diff, normalized_not_wugs_far_diff = get_normalized_diff_to_mean_wug(p, tp, tn_hard, tn_easy)
        
        hard_accs.append(acc_for_threshold(normalized_wugs_diff_by_close, normalized_not_wugs_close_diff, max_t))
        easy_accs.append(acc_for_threshold(normalized_wugs_diff_by_far, normalized_not_wugs_far_diff, max_t))
        
    return hard_accs, easy_accs
    
    




In [5]:
auc_low_hard, auc_low_easy, auc_high_hard, auc_high_easy = [], [], [], []
for (pl, ph, tpl, tph, tnlh, tnhh, tnle, tnhe) in zip(train_pos_low, train_pos_high, test_pos_low, test_pos_high, test_neg_low_hard, test_neg_high_hard, test_neg_low_easy, test_neg_high_easy):
    c_auc_low_hard, c_auc_low_easy, c_auc_high_hard, c_auc_high_easy = eval_for_task(pl, ph, tpl, tph, tnlh, tnhh, tnle, tnhe)
    
    auc_low_hard.append(c_auc_low_hard)
    auc_low_easy.append(c_auc_low_easy)   
    auc_high_hard.append(c_auc_high_hard)
    auc_high_easy.append(c_auc_high_easy)

In [6]:
acc_t_all_low_hard, acc_t_all_low_easy = eval_max_acc(train_pos_low, test_pos_low, test_neg_low_hard, test_neg_low_easy)
acc_t_all_high_hard, acc_t_all_high_easy = eval_max_acc(train_pos_high, test_pos_high, test_neg_high_hard, test_neg_high_easy)


